In [26]:
import os
import cv2
import numpy as np

In [27]:
def preprocess_frame(frame):
    # Apply filtering, smoothing, sharpening, and adjusting brightness and contrast
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(blurred, -1, kernel)
    alpha = 1.1  # Contrast control
    beta = 15    # Brightness control
    adjusted = cv2.convertScaleAbs(sharpened, alpha=alpha, beta=beta)
    
    # Normalize the image
    normalized = cv2.normalize(adjusted, None, 0, 255, cv2.NORM_MINMAX)
    
    return normalized

def save_frames_as_avi(frames, output_path, fps=30):
    if not frames:
        print("No frames to save.")
        return
    
    height, width, layers = frames[0].shape
    size = (width, height)
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    
    for frame in frames:
        out.write(frame)
    
    out.release()
    print(f"Video saved as {output_path}")

In [28]:
def ucf50_preprocessing(data_dir, output_dir, class_names, number_of_frames, threshold=60):
    orb = cv2.ORB_create()
    for class_name in class_names:
        class_input_path = os.path.join(data_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)
        
        for video_name in os.listdir(class_input_path):
            video_path = os.path.join(class_input_path, video_name)
            cap = cv2.VideoCapture(video_path)
            relevant_frames = []
            all_frames = []
            prev_descriptors = None
            total_frames = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                total_frames += 1
                all_frames.append(frame)
                processed_frame = preprocess_frame(frame)
                keypoints, descriptors = orb.detectAndCompute(processed_frame, None)
                
                if descriptors is not None and len(descriptors) > 0:
                    if prev_descriptors is None:
                        relevant_frames.append(frame)
                    else:
                        # Compute the distance between descriptors
                        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                        matches = bf.match(prev_descriptors, descriptors)
                        distances = [m.distance for m in matches]
                        avg_distance = np.mean(distances)
                        
                        if avg_distance > threshold:
                            relevant_frames.append(frame)
                    
                    prev_descriptors = descriptors
            
            cap.release()
            
            # Ensure minimum number of frames is at least 10
            if len(relevant_frames) < number_of_frames:
                needed_frames = number_of_frames - len(relevant_frames)
                relevant_frames.extend(all_frames[:needed_frames])
            
            print(f"Video: {video_name}")
            print(f"Total frames: {total_frames}")
            print(f"Selected frames: {len(relevant_frames)}\n")
            
            # Save the selected frames as an AVI video
            output_video_path = os.path.join(output_class_path, video_name)
            save_frames_as_avi(relevant_frames, output_video_path)

In [ ]:
data_dir = '../data/raw/UCF50'

In [ ]:
class_names = os.listdir(data_dir)

In [ ]:
output_dir = '../data/processed/ORB'

In [ ]:
ucf50_preprocessing(data_dir, output_dir, class_names, number_of_frames=30, threshold=50)